In [1]:
from datasets import load_data, aggregate
import matplotlib.pyplot as plt 
import pandas as pd 
import settings
import os
import pickle
import timeit 
import dask.dataframe as dd

BASEDIR = os.path.join(os.getcwd(), '..', 'data', 'clean', 'all-data.pkl')
file = open(BASEDIR, 'rb')

In [ ]:
start = timeit.timeit()
df = pd.DataFrame(pickle.load(file))
aggdf = df.groupby(['year', 'state']).agg({'payment': 'sum'})
end = timeit.timeit()
print(aggdf.head())
print("{:.2f}".format(end - start))

In [2]:
start = timeit.timeit()
df = dd.from_pandas(pickle.load(file))
aggdf = df.groupby(['year', 'state']).agg({'payment': 'sum'})
end = timeit.timeit()
print(aggdf.head())
print("{:.2f}".format(end - start))

ValueError: Exactly one of npartitions and chunksize must be specified.

In [ ]:
summarytable = aggdf.groupby(['year']).apply(lambda x: x.nlargest(5, 'payment'))
summarytable.drop(columns='year', inplace=True)
summarytable.index = summarytable.index.droplevel(level=1)
summarytable.set_index('state', append=True, inplace=True)
summarytable
# # summarytable['payment'] = summarytable['payment'].apply(lambda x: "${:,.2f}M".format((x/1000000)))
# summarytable = summarytable.merge(cnty_state_crswlk, how='left')
# # summarytable.drop(columns=['year'], inplace=True) 
# summarytable.set_index('year', inplace=True)
# summarytable.drop(columns=['FIP'], inplace=True)
# summarytable.rename(columns = {'cntyState': 'County'}, inplace=True)
# summarytable.set_index('County', append=True, inplace=True)
# print(summarytable)
# summarytable.join(df, by=)
# fname = 'test_file.tex'
# file = os.path.join(os.getcwd(), '..', 'output', 'tables', fname)
# # summarytable.to_latex(float_format=lambda x: "\$ {:,.2f} M".format(x/1000000))

# with open(file, 'w') as f: 
#     f.write(summarytable.to_latex(
#         float_format=lambda x: "\$ {:,.2f} M".format(x/1000000)
#     ))

In [ ]:
test = df.drop_duplicates(subset=['FIP'])[['FIP', 'county', 'state']]

test['cntyState'] = test.apply(lambda x: f"{x['county']}, {x['state']}", axis=1)
test.drop(columns=['county','state'], inplace=True)
test